# Final Submission <br>
Final submission for NLP Corsework 


## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Pre Procesing

In [3]:
from pre_processing import PreProcessing
pre_process = PreProcessing()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
reviews = pd.read_csv('reviews.csv')

In [5]:
processed = pre_process.set_n_grams(2, pre_process.lemmatize(pre_process.lowercase(pre_process.remove_stopwords(pre_process.remove_punctuation(reviews.review)))))

## TFIDF

In [6]:
from tfidf_class_to_analyze import TFIDF
tfidf = TFIDF()

In [7]:
review_terms = {}
label_terms = []
for review_id in range(len(processed)):
    review_terms[review_id] = tfidf.get_terms(processed[review_id])


all_terms = tfidf.collect_vocabulary(review_terms)

#doc_vectors = tfidf.vectorize(review_terms, all_terms)

doc_idfs = tfidf.calculate_idfs(all_terms, review_terms)
#doc_vectors = tfidf.vectorize_idf(review_terms, doc_idfs, all_terms)

In [19]:
doc_vectors = tfidf.vectorize_idf(review_terms, doc_idfs, all_terms)

In [9]:
pruned_features = pre_process.remove_low_proba(doc_idfs, 0.05)

AxisError: axis -1 is out of bounds for array of dimension 0

In [10]:
len(doc_vectors[0])

42293

In [12]:
features = np.array(list(doc_vectors.values()))

proba_features = np.sum(features, axis = 0) / np.sum(features)

sorted_indexes = np.argsort(proba_features)
sorted_probs = np.sort(proba_features)

cumulative_sum = np.cumsum(sorted_probs)

remove_point = 0.05 * np.sum(proba_features)

to_remove_sorted = np.argmax(cumulative_sum >= remove_point)

indexes = sorted_indexes[to_remove_sorted:]



In [21]:
sorted_indexes[to_remove_sorted:]

array([41047, 41049,  7419, ..., 24735, 14071,  5100], dtype=int64)

## Train, Test, Dev Splits

In [11]:
X_train, X_test, X_dev, y_train, y_test, y_dev = pre_process.set_data_splits(pruned_features, list(reviews.label))

In [13]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Naive Bayes

In [14]:
from nb_classifier_2 import NB_Classifier
nb2 = NB_Classifier(alpha=0.8)

In [29]:
X = np.array(X_train)

In [15]:
nb2.fit(X_train, y_train)

In [16]:
pred = nb2.predict(X_train)

In [17]:
pred_test = nb2.predict(np.array(X_test))

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming 'y_true' is the true labels and 'y_pred' is the predicted labels
print("Confusion Matrix:\n", confusion_matrix(y_train, pred))
print("Accuracy:", accuracy_score(y_train, pred))
print("Precision:", precision_score(y_train, pred))
print("Recall:", recall_score(y_train, pred))
print("F1 Score:", f1_score(y_train, pred))


Confusion Matrix:
 [[1422    0]
 [ 293 1085]]
Accuracy: 0.8953571428571429
Precision: 1.0
Recall: 0.7873730043541364
F1 Score: 0.881039382866423


In [21]:
pred2 = mnb.predict(X_train)

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming 'y_true' is the true labels and 'y_pred' is the predicted labels
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_test))
print("Accuracy:", accuracy_score(y_test, pred_test))
print("Precision:", precision_score(y_test, pred_test))
print("Recall:", recall_score(y_test, pred_test))
print("F1 Score:", f1_score(y_test, pred_test))


Confusion Matrix:
 [[ 56 245]
 [ 72 227]]
Accuracy: 0.4716666666666667
Precision: 0.4809322033898305
Recall: 0.7591973244147158
F1 Score: 0.5888456549935148


In [20]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, td)
pred = mnb.predict(X_train)

# Assuming 'y_true' is the true labels and 'y_pred' is the predicted labels
print("Confusion Matrix:\n", confusion_matrix(td, pred))
print("Accuracy:", accuracy_score(td, pred))
print("Precision:", precision_score(td, pred))
print("Recall:", recall_score(td, pred))
print("F1 Score:", f1_score(td, pred))

ValueError: Found input variables with inconsistent numbers of samples: [2800, 600]

## Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

outp= log_reg.predict_proba(X_train)


In [ ]:
outp= log_reg.predict_proba(X_test)

In [19]:
predicts = []
for i in range(len(outp)):
    if outp[i][0] < outp[i][1]: predicts.append(1)
    else :predicts.append(0)


In [21]:
print("Confusion Matrix:\n", confusion_matrix(y_train, predicts))
print("Accuracy:", accuracy_score(y_train, predicts))
print("Precision:", precision_score(y_train, predicts))
print("Recall:", recall_score(y_train, predicts))
print("F1 Score:", f1_score(y_train, predicts))

Confusion Matrix:
 [[1422    0]
 [   0 1378]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


## Bert

In [7]:
X_train, X_test, X_dev, y_train, y_test, y_dev = pre_process.set_data_splits(reviews.review, reviews.label)

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_dev), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

In [16]:

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, list(y_train))
val_dataset = IMDbDataset(val_encodings, list(y_dev))
test_dataset = IMDbDataset(test_encodings, list(y_test))

In [22]:
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device(type='cuda', index=0)

TypeError: device() received an invalid combination of arguments - got (), but expected one of:
 * (torch.device device)
 * (str type, int index)


In [24]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    no_cuda= False
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to('cuda')

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset

)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/525 [00:00<?, ?it/s]

{'loss': 0.7087, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.06}
{'loss': 0.7079, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.11}
{'loss': 0.7217, 'learning_rate': 3e-06, 'epoch': 0.17}
{'loss': 0.6852, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.23}
{'loss': 0.6813, 'learning_rate': 5e-06, 'epoch': 0.29}
{'loss': 0.6638, 'learning_rate': 6e-06, 'epoch': 0.34}
{'loss': 0.6478, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.4}
{'loss': 0.6207, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.46}
{'loss': 0.5881, 'learning_rate': 9e-06, 'epoch': 0.51}
{'loss': 0.5614, 'learning_rate': 1e-05, 'epoch': 0.57}
{'loss': 0.556, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.63}
{'loss': 0.3925, 'learning_rate': 1.2e-05, 'epoch': 0.69}
{'loss': 0.3736, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.74}
{'loss': 0.4339, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.8}
{'loss': 0.3734, 'learning_rate': 1.5e-05, 'epoch': 0.86}
{'loss': 0.3617, 'learning

TrainOutput(global_step=525, training_loss=0.3391809720084781, metrics={'train_runtime': 6857.364, 'train_samples_per_second': 1.225, 'train_steps_per_second': 0.077, 'train_loss': 0.3391809720084781, 'epoch': 3.0})

In [25]:
device

device(type='cuda', index=0)

In [19]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
trainer.train()

  0%|          | 0/350 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.28091785311698914, 'eval_accuracy': 0.9, 'eval_runtime': 25.9731, 'eval_samples_per_second': 23.101, 'eval_steps_per_second': 1.463, 'epoch': 1.0}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.3185114562511444, 'eval_accuracy': 0.9, 'eval_runtime': 26.1432, 'eval_samples_per_second': 22.951, 'eval_steps_per_second': 1.454, 'epoch': 2.0}
{'train_runtime': 4686.3677, 'train_samples_per_second': 1.195, 'train_steps_per_second': 0.075, 'train_loss': 0.3038702828543527, 'epoch': 2.0}


TrainOutput(global_step=350, training_loss=0.3038702828543527, metrics={'train_runtime': 4686.3677, 'train_samples_per_second': 1.195, 'train_steps_per_second': 0.075, 'train_loss': 0.3038702828543527, 'epoch': 2.0})

In [22]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [28]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [29]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="stevhliu/my_awesome_model")
classifier(text)

config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

c:\Users\giova\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\giova\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.9994940757751465}]

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_model")
inputs = tokenizer(text, return_tensors="pt")

In [31]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [32]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'

In [34]:
predicts = []
for w in list(X_train):
    inputs = tokenizer(w, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    predicts.append(1 if model.config.id2label[predicted_class_id] == 'LABEL_1' else 0)

RuntimeError: The size of tensor a (747) must match the size of tensor b (512) at non-singleton dimension 1

In [36]:
for w in X_test:
    classifier(w)

((NB: Spoiler warning, such as it is!))<br /><br />First off, this is a teen slasher flick -- the Spam-In-A-Cabin genre, as Joe Bob Briggs piquantly put it. If you're looking for Roshambo, this isn't it and wasn't going to BE it. I'm desperately unimpressed by stabs at its cinematography, directing or acting performances.<br /><br />Secondly, this wasn't Zuniga's first horror flick, it was her first screen appearance period, cinema, TV, whatever. For what it is worth; neither is Daphne Zuniga Susan Sarandon or Katherine Hepburn.<br /><br />Thirdly, you have to give even a lame slasher flick props. Sure, it follows the deeply insulting formulaic message of its genre: any young woman having or showing interest in sex is beef on the hoof, and the harvest time is now.<br /><br />Except this one gives the chop to the sweet, virginal protagonist as well! Now THERE is a mediocre teen death film that has the courage of its convictions! Interesting that this was said ingenue's only film role. A